In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from time import gmtime, strftime 
from pylab import rcParams
import statsmodels.api as sm
import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX
from itertools import product
#from statsmodels.tsa.statespace.sarimax import seasonal_decompose

In [ ]:
invoice = pd.read_csv("Final_invoice.csv",low_memory=False)

# Model Building

In [ ]:
invoice['Month-Year'] = pd.to_datetime(invoice['Invoice Date']).dt.strftime('20%y-%m')

In [ ]:
revenue_model = invoice.groupby(['Model','Month-Year']).agg({'Total Amt Wtd Tax.':['mean','count']})

In [ ]:
revenue_model

In [ ]:
revenue_model.columns = revenue_model.columns.droplevel()

# Revenue = Mean * Count

In [ ]:
revenue_model['Revenue'] = revenue_model['mean'] * revenue_model['count']

In [ ]:
revenue_model

In [ ]:
model_data = revenue_model.round(1).pivot_table(index = ['Month-Year'], columns = ['Model'], values = ['Revenue'], fill_value=0, aggfunc='sum').reset_index()

In [ ]:
model_data.head()

In [ ]:
model_data.columns = model_data.columns.droplevel()

In [ ]:
model_data.rename(columns={'':'Month-Year'},inplace=True)

In [ ]:
model_data.head()

In [ ]:
model_data['Total_Revenue'] =  model_data.iloc[:,1:].sum(axis=1)

In [ ]:
model_data.head()

In [ ]:
model_data.to_csv('model data.csv')

In [ ]:
Time_Series_data = model_data[['Total_Revenue','Month-Year']].sort_values(by = 'Month-Year')
Time_Series_data['Month-Year'] = pd.to_datetime(Time_Series_data['Month-Year'])
Time_Series_data.set_index('Month-Year',inplace=True)

In [ ]:
Time_Series_data

In [ ]:
result = sm.tsa.seasonal_decompose(Time_Series_data['Total_Revenue'], model='multiplicative')

In [ ]:
result.plot()

In [ ]:
x =  Time_Series_data['Total_Revenue'].values

In [ ]:
p=d=q=range(0,10)

In [ ]:
pdq=list(product(p,d,q))

In [ ]:
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in pdq]

In [ ]:
aic_list = {}
for param in pdq:
    try:
        model_arima = ARIMA()

In [ ]:
#for param in pdq:
#    for param_seasonal in seasonal_pdq:
#        try:
#            model = sm.tsa.statespace.SARIMAX(x, order=param, seasonal_order=param_seasonal, enforce_stationarity=False,
#                                            enforce_invertibility=False)
#            results = model.fit()
#            print('ARIMA{}x{} - AIC:{}'.format(param, param_seasonal, results.aic))
#        except:
#            continue

In [ ]:
model = sm.tsa.statespace.SARIMAX(y,
                                  order = (0, 0, 0),
                                  seasonal_order = (0, 7, 0, 12),
                                  enforce_stationarity=False,
                                            enforce_invertibility=False)

In [ ]:
results = model.fit()